# III. Inference: 모델 로드 및 추론 예

실제 프로젝트 방식 vs. 채점 편의:
- 실제 프로젝트 방식: 
    - colab 환경에서 진행하여서 Hugging Face 로그인 과정을 별도의 cell에 구현하였습니다.
    - inference data 접근 및 result 저장을 위하여 전체 리포지토리를 clone하는 과정도 별도의 cell에 구현되어있습니다.
- **💡채점 편의💡**: 
    - 로컬 환경을 고려하여 Hugging Face 로그인 과정 cell이 별도로 분리되어 있습니다.
    - 추론 데이터 샘플을 코드 내부에 **하드코딩하는 셀을 별도로 추가**하여, repository 복제 없이도 즉시 추론 결과를 확인할 수 있도록 구성하였습니다. (4. 추론 예시 실행 파트의 첫 번째 cell)
    - 하드코딩 방식에서는 result 파일 저장을 생략하였습니다 (실제 방식: 추론 결과 datas/inference_result.csv에 저장)

## 1. 환경 설정 및 필수 라이브러리 로드



In [ ]:
# =========================================================
# git clone
# =========================================================

# 1. 깃 클론 및 경로 이동 (Test Set 파일 접근 및 result 파일 저장)
!git clone https://github.com/jsjang0104/HUFS-LAI-ML-2025-2.git
!cd HUFS-LAI-ML-2025-2
print("✅ 깃 클론 및 경로 이동 완료.")

: 

In [ ]:
# =========================================================
# libaray install, hugging face login
# =========================================================

# 1. transformers, datasets
%pip install transformers==4.27.4 datasets -q
print("✅ transformers와 datasets 재설치 완료")

# 2. Hugging Face 로그인
from huggingface_hub import notebook_login
notebook_login()
print("✅ 허깅페이스 로그인 완료")

Note: you may need to restart the kernel to use updated packages.
✅ transformers와 datasets 재설치 완료


You should consider upgrading via the 'c:\Users\JisooJang\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [1]:
# =========================================================
# 라이브러리 import
# =========================================================
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
print("✅ 모든 라이브러리 import 완료")

c:\Users\JisooJang\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 모든 라이브러리 import 완료


In [2]:
# =========================================================
# 하이퍼파라미터, 경로 설정
# =========================================================
HF_MODEL_ID = "jsjang0104/book-genre-classifier-bert"
LABELS = ['Geschichte', 'Literatur', 'Sozialwissenschaften', 'Sprachwissenschaft'] #CLASS_NAMES
NUM_CLASSES = len(LABELS)
MAX_LEN = 256 # training.ipynb와 동일
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("✅ 하이퍼파라미터 및 경로 설정 완료")

✅ 하이퍼파라미터 및 경로 설정 완료


## 2. 모델 로드 및 복원

In [3]:
print("--- 1. 모델 로드 시작 ---")
try:
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_ID)

    # 가중치 파일 로드 및 모델 복원
    model = AutoModelForSequenceClassification.from_pretrained(HF_MODEL_ID, num_labels=NUM_CLASSES, trust_remote_code=True)
    model.to(device)
    model.eval() # 모델을 평가 모드로 설정

    print(f"✅ 모델 '{HF_MODEL_ID}' 로드 완료 (Device: {device})")

except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    print("GitHub/Hugging Face ID 또는 네트워크 상태를 확인하세요.")
    print(f" [에러 해결 안내]")
    print(f"   - **원인:** Hugging Face의 최신 가중치 파일 형식인 'model.safetensors'를 로드하는 데 필요한 ")
    print(f"     라이브러리가 로컬 환경에 설치되어 있지 않습니다.")
    print(f"   - **해결 방법 (가장 유력):** 터미널/프롬프트에서 아래 명령어를 실행하여 'safetensors' 라이브러리를 설치하세요.")
    print(f"     >>> pip install safetensors")
    print(f"   - **추가 해결 방법:** 위 조치 후에도 오류가 지속되면 'transformers' 라이브러리를 최신 버전으로 업데이트하세요.")
    print(f"     >>> pip install --upgrade transformers")
    exit()

--- 1. 모델 로드 시작 ---
✅ 모델 'jsjang0104/book-genre-classifier-bert' 로드 완료 (Device: cpu)


c:\Users\JisooJang\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JisooJang\.cache\huggingface\hub\models--jsjang0104--book-genre-classifier-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## 3. 추론 함수 정의

In [4]:
def predict_genre(title: str):
    """
    주어진 제목에 대해 장르를 예측하고 상위 4개 클래스의 확률을 반환하는 함수
    """

    # 토크나이징 (기존 로직 유지)
    encoding = tokenizer.encode_plus(
        title,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    # 디바이스 이동 및 추론 (기존 로직 유지)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # 확률 변환
    probs = F.softmax(logits, dim=1).cpu().numpy()[0]

    # 확률 높은 순서대로 인덱스 정렬
    ranked_indices = np.argsort(probs)[::-1]

    # 상위 4개 클래스 이름과 확률을 딕셔너리로 저장
    top_4_probs = {}
    for i in range(min(4, NUM_CLASSES)):
        idx = ranked_indices[i]
        class_name = LABELS[idx]
        top_4_probs[class_name] = probs[idx]

    # 최종 예측 (highest confidence)
    prediction = LABELS[ranked_indices[0]]
    confidence = probs[ranked_indices[0]]

    return prediction, confidence, top_4_probs

## 4. 추론 예시 실행
- 오스트리아 도서관에서 아직 labeling이 되지 않은 도서 목록에 대해서 시행
    - 각 class (어학, 사회과학, 문학, 역사) 별로 5권 씩 선정
    - 영어, 독일어, 한국어, 라틴어 등 다국어로 선정





In [ ]:
# ------------------------------------------------------------------------------------------------
# 💡 하드코딩 방식 💡
# ------------------------------------------------------------------------------------------------

hardcoded_data = [
    {"title": "A study on tones and tonemarks in middle korean", "subject": "Sprachwissenschaft"},
    {"title": "Einführung in die Linguistik", "subject": "Sprachwissenschaft"},
    {"title": "Studien zur Texttheorie und zur deutschen Grammatik", "subject": "Sprachwissenschaft"},
    {"title": "ABC der schwachen Verben", "subject": "Sprachwissenschaft"},
    {"title": "라틴어 문법", "subject": "Sprachwissenschaft"},
    {"title": "Austrian foreign policy yearbook", "subject": "Sozialwissenschaften"},
    {"title": "Tatsachen und Zahlen", "subject": "Sozialwissenschaften"},
    {"title": "Außenpolitischer Bericht", "subject": "Sozialwissenschaften"},
    {"title": "Wirtschaftspolitik und Arbeitsmarkt", "subject": "Sozialwissenschaften"},
    {"title": "독일 민법전", "subject": "Sozialwissenschaften"},
    {"title": "Emblematium Libellus", "subject": "Literatur"},
    {"title": "Brechts Antigone des Sophokles", "subject": "Literatur"},
    {"title": "Der Mantel des Darius", "subject": "Literatur"},
    {"title": "1001 Nacht. Arabische Erzählung mit 63 Illustrationen", "subject": "Literatur"},
    {"title": "셰익스피어 비극론", "subject": "Literatur"},
    {"title": "Lexikon der antiken Mythen und Gestalten", "subject": "Geschichte"},
    {"title": "Der römische Staat I. Die Republik", "subject": "Geschichte"},
    {"title": "BIBLIOTHEK DER GESCHICHTE UND POLITIK DIE FRAN...", "subject": "Geschichte"},
    {"title": "서양미술사", "subject": "Geschichte"},
    {"title": "한국외국어대학교 60년사 I. 시대별 외대사", "subject": "Geschichte"},
]

new_titles = [(item['title'], item['subject']) for item in hardcoded_data]
print(f"✅ inference data {len(new_titles)}개 준비 완료 (hardcoded)")

results = []
for title, actual_subject in new_titles:
    if not isinstance(title, str) or title.strip() == "":
        continue

    prediction, confidence, top_4_probs = predict_genre(title)

    results.append({
        'Title': title,
        'Actual Subject': actual_subject,
        'Predicted Genre': prediction,
        'Confidence': f"{confidence:.4f}",
        'Top 4 Probabilities_Dict': top_4_probs,
        'Confidence_Value': confidence
    })

# 결과 출력
results_df = pd.DataFrame(results)

print("\n--- 추론 결과 요약 ---")
print(results_df[['Title', 'Predicted Genre', 'Confidence', 'Actual Subject']])

print("\n--- 상위 4개 확률 분포 상세 ---")
for result in results:
    print(f"\nTitle: {result['Title']}")
    print(f"  -> Actual: {result['Actual Subject']}")
    print(f"  -> Predicted: {result['Predicted Genre']} (Conf: {result['Confidence']})")
    print(f"  -> Top 4 Probabilities:")

    top_4_dict = result['Top 4 Probabilities_Dict']
    for genre, prob in top_4_dict.items():
        print(f"    - {genre}: {prob:.4f}")

✅ inference data 20개 준비 완료 (hardcoded)

--- 추론 결과 요약 ---
                                                Title       Predicted Genre  \
0     A study on tones and tonemarks in middle korean    Sprachwissenschaft   
1                        Einführung in die Linguistik    Sprachwissenschaft   
2   Studien zur Texttheorie und zur deutschen Gram...    Sprachwissenschaft   
3                            ABC der schwachen Verben             Literatur   
4                                              라틴어 문법    Sprachwissenschaft   
5                    Austrian foreign policy yearbook  Sozialwissenschaften   
6                                Tatsachen und Zahlen  Sozialwissenschaften   
7                            Außenpolitischer Bericht  Sozialwissenschaften   
8                 Wirtschaftspolitik und Arbeitsmarkt  Sozialwissenschaften   
9                                              독일 민법전  Sozialwissenschaften   
10                               Emblematium Libellus             Literatu

In [ ]:
# ------------------------------------------------------------------------------------------------
# 실제 방식
# ------------------------------------------------------------------------------------------------
INFERENCE_DATA_PATH = '/content/HUFS-LAI-ML-2025-2/submissions/202400420/assignment5/eval_dataset/inference_data.csv'
OUTPUT_CSV_PATH = '/content/HUFS-LAI-ML-2025-2/submissions/202400420/assignment5/eval_dataset/inference_results.csv'

try:
    df_inference = pd.read_csv(INFERENCE_DATA_PATH)
    new_titles = df_inference['title'].tolist()
    print(f"✅ 추론 데이터 {len(new_titles)}개 로드 완료")
except FileNotFoundError:
    print(f"'파일을 찾을 수 없습니다. 경로를 확인해주세요")
    exit()

# 추론
results = []
for title in new_titles:
    if not isinstance(title, str) or title.strip() == "":
        continue

    prediction, confidence, top_4_probs = predict_genre(title)

    actual_subject = df_inference[df_inference['title'] == title]['subject'].iloc[0] if 'subject' in df_inference.columns else ''

    results.append({
        'Title': title,
        'Actual Subject': actual_subject,
        'Predicted Genre': prediction,
        'Confidence': f"{confidence:.4f}",
        'Top 4 Probabilities_Dict': top_4_probs, 
        'Confidence_Value': confidence 
    })

# 결과 출력 및 저장
results_df = pd.DataFrame(results)
results_df['Top 4 Probabilities_CSV'] = results_df['Top 4 Probabilities_Dict'].apply(str)

try:
    # 딕셔너리가 아닌 문자열 컬럼을 사용하여 저장
    results_df[['Title', 'Actual Subject', 'Predicted Genre', 'Confidence', 'Top 4 Probabilities_CSV']].to_csv(
        OUTPUT_CSV_PATH, index=False, encoding='utf-8'
    )
    print(f"\n✅ 추론 결과 {len(results_df)}개가 '{OUTPUT_CSV_PATH}' 파일로 저장 완료되었습니다.")
except Exception as e:
    print(f"\n❌ CSV 파일 저장 중 오류 발생: {e}")

print("--- 추론 결과 요약 ---")
print(results_df[['Title', 'Predicted Genre', 'Confidence', 'Actual Subject']])

print("--- 상위 4개 확률 분포 상세 ---")
for result in results:
    print(f"\nTitle: {result['Title']}")
    print(f"  -> Predicted: {result['Predicted Genre']} (Conf: {result['Confidence']})")
    print(f"  -> Top 4 Probabilities:")

    # 딕셔너리 객체 ('Top 4 Probabilities_Dict')를 사용하여 출력
    top_4_dict = result['Top 4 Probabilities_Dict']
    for genre, prob in top_4_dict.items():
        print(f"     - {genre}: {prob:.4f}")